# GaitRehab Explanatory Data Analysis

In [2]:
# Libraries
import os, sys, math, random, time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Constants
data_path = "/Users/stamatiosorphanos/Documents/GaitRehab_Models/data"

## Inertial Measurement Unit - IMU EDA

1. ELT process and standarization of data format
2. Plot data and provide explorations


In [5]:
healthy_directory = os.path.join(data_path, "Healthy")
stroke_directory  = os.path.join(data_path, "Stroke")

for subdir, dirs, files in os.walk(healthy_directory):
    print(subdir)
    for file in files:
        file_path = os.path.join(healthy_directory, file)
        formatted_name = file.replace(" ", "_").replace("-", "_")
        print(formatted_name)
        date = file.split("T")[0]
        print(date)
        new_name = file.split("_")[0] +  ".csv"     
    

/Users/stamatiosorphanos/Documents/GaitRehab_Models/data/Healthy
/Users/stamatiosorphanos/Documents/GaitRehab_Models/data/Healthy/Patient_1
Vicon_Trial02.csv
Vicon_
/Users/stamatiosorphanos/Documents/GaitRehab_Models/data/Healthy/Patient_1/IMU
E3_2022_01_19T10.53.38.171_FB7D6148EEE3_Accelerometer.csv
E3-2022-01-19
Meta4_7D_2022_01_19T10.53.38.171_D2DC12F387FB_Gyroscope.csv
Meta4 7D-2022-01-19
CF_2022_01_19T10.53.38.171_D26534B09ECF_Accelerometer.csv
CF-2022-01-19
Meta1_2022_01_19T10.53.38.171_D71B3D900A76_Gyroscope.csv
Meta1-2022-01-19
Meta1_2022_01_19T10.53.38.171_D71B3D900A76_Accelerometer.csv
Meta1-2022-01-19
3D_2022_01_19T10.53.38.171_F0794D73E83D_Accelerometer.csv
3D-2022-01-19
CF_2022_01_19T10.53.38.171_D26534B09ECF_Gyroscope.csv
CF-2022-01-19
MetaWear_2022_01_19T10.53.38.171_D70A7F8252C8_Accelerometer.csv
MetaWear-2022-01-19
E3_2022_01_19T10.53.38.171_FB7D6148EEE3_Gyroscope.csv
E3-2022-01-19
Meta4_7D_2022_01_19T10.53.38.171_D2DC12F387FB_Accelerometer.csv
Meta4 7D-2022-01-19
3D_2